# SQL 
Responsable: Juan Alejandro Gómez Jaramillo

Repositorio: https://github.com/agomezj4/inteligencia_digital

In [19]:
import chardet
import pandas as pd
import re

In [51]:
ruta = '/Users/agomezj/Desktop/Data Science/Proyectos/inteligencia_digital/sql/data/raw/RUAF.csv'
ruta2 = '/Users/agomezj/Desktop/Data Science/Proyectos/inteligencia_digital/sql/data/raw/muestra_RUAF.xlsx'

In [53]:
prueba = pd.read_excel(ruta2)

In [4]:
# 1. Identificamos el encoding del archivo
with open(ruta, 'rb') as f:
    result = chardet.detect(f.read())
    print(result)

{'encoding': 'ISO-8859-1', 'confidence': 0.7298615696672321, 'language': ''}


In [13]:
# 2. Cargamos el archivo
with open(ruta, 'r', encoding='ISO-8859-1') as file:
    lines = file.readlines()

In [35]:
#3. Dividir el texto plano por registros

# Inicializar lista para almacenar registros
registros = []
registro_actual = []

# Procesar cada línea
for line in lines:
    # Verificar si la línea indica el inicio de un nuevo registro
    if line.startswith('¬-*-¬DATA'):
        if registro_actual:
            # Agregar el registro actual a la lista de registros
            registros.append(registro_actual)
        # Iniciar un nuevo registro
        registro_actual = []
    # Añadir la línea actual al registro actual
    registro_actual.append(line.strip())  # Utiliza strip() para eliminar espacios en blanco alrededor de la línea

# Agregar el último registro a la lista si no está vacío
if registro_actual:
    registros.append(registro_actual)

In [87]:
registros[0]

['¬-*-¬DATA;;2021-09-09 17:14:36',
 '¬-*-¬ID;3946612;2021-09-09 17:14:36',
 '5|CC;1116912531;VINCULACIÓN A PROGRAMAS DE  ASISTENCIA SOCIAL;;2021-09-09 17:14:36',
 '5|CC;1116912531;PENSIONADOS;;2021-09-09 17:14:36',
 '5|CC;1116912531;AFILIACIÓN A CESANTIAS;;2021-09-09 17:14:36',
 '5|CC;1116912531;AFILIACIÓN A RIESGOS LABORALES;;2021-09-09 17:14:36',
 '5|CC;1116912531;INFORMACIÓN BASICA;Segundo Apellido;M;2021-09-09 17:14:36',
 '5|CC;1116912531;INFORMACIÓN BASICA;Primer Apellido;BEDOYA;2021-09-09 17:14:36',
 '5|CC;1116912531;INFORMACIÓN BASICA;Segundo Nombre;MEJIA;2021-09-09 17:14:36',
 '5|CC;1116912531;INFORMACIÓN BASICA;Primer Nombre;MAURICIO;2021-09-09 17:14:36',
 '5|CC;1116912531;INFORMACIÓN BASICA;Número de Identificación;CC 1116912531;2021-09-09 17:14:36',
 '5|CC;1116912531;AFILIACIÓN A SALUD;Departamento -> Municipio;FLORENCIA;2021-09-09 17:14:36',
 '5|CC;1116912531;AFILIACIÓN A SALUD;Tipo de Afiliado;COTIZANTE;2021-09-09 17:14:36',
 '5|CC;1116912531;AFILIACIÓN A SALUD;Estado de A

In [123]:
#4. Por medio de expresiones regualres extraemos inf de cada campo
patrones = {
    'key_fep': re.compile(r'¬-*-¬ID;(\d+);'),
    'origen': None,  # Asignar directamente 'RUAF'
    'f_consulta': re.compile(r'¬-*-¬DATA;;(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})'),
    'num_id': re.compile(r'INFORMACIÓN BASICA;Número de Identificación;(\d+);'),
    'tipo_id_str': re.compile(r'INFORMACIÓN BASICA;Número de Identificación;([A-Z]+) (\d+);'),  # Extraer tipo de identificación
    'primer_nombre': re.compile(r'INFORMACIÓN BASICA;Primer Nombre;(.*?);'),
    'segundo_nombre': re.compile(r'INFORMACIÓN BASICA;Segundo Nombre;(.*?);'),
    'primer_apellido': re.compile(r'INFORMACIÓN BASICA;Primer Apellido;(.*?);'),
    'segundo_apellido': re.compile(r'INFORMACIÓN BASICA;Segundo Apellido;(.*?);'),
    'sexo': re.compile(r'INFORMACIÓN BASICA;Sexo;(.*?);'),
    'admon_salud': re.compile(r'AFILIACIÓN A SALUD;Administradora;(.*?);'),
    'regimen_salud': re.compile(r'AFILIACIÓN A SALUD;Régimen;(.*?);'),
    'f_afiliacion_salud': re.compile(r'AFILIACIÓN A SALUD;Fecha Afiliacion;(.*?);'),
    'est_afiliacion_salud': re.compile(r'AFILIACIÓN A SALUD;Estado de Afiliación;(.*?);'),
    'tipo_afiliado_salud': re.compile(r'AFILIACIÓN A SALUD;Tipo de Afiliado;(.*?);'),
    'dep_municipio': re.compile(r'AFILIACIÓN A SALUD;Departamento -> Municipio;(.*?);'),
    'regimen_pension': re.compile(r'AFILIACIÓN A PENSIONES;Régimen;(.*?);'),
    'admon_pension': re.compile(r'AFILIACIÓN A PENSIONES;Administradora;(.*?);'),
    'f_afiliacion_pension': re.compile(r'AFILIACIÓN A PENSIONES;Fecha de Afiliación;(.*?);'),
    'est_afiliacion_pension': re.compile(r'AFILIACIÓN A PENSIONES;Estado de Afiliación;(.*?);'),
    'admon_rl': re.compile(r'AFILIACIÓN A RIESGOS LABORALES;Administradora;(.*?);'),
    'f_afiliacion_rl': re.compile(r'AFILIACIÓN A RIESGOS LABORALES;Fecha de Afiliación;(.*?);'),
    'est_afiliacion_rl': re.compile(r'AFILIACIÓN A RIESGOS LABORALES;Estado de Afiliación;(.*?);'),
    'act_economica_rl': re.compile(r'AFILIACIÓN A RIESGOS LABORALES;Actividad Economica;(.*?);'),
    'municipio_laboral_rl': re.compile(r'AFILIACIÓN A RIESGOS LABORALES;Municipio Labora;(.*?);'),
    'admon_cf': re.compile(r'AFILIACIÓN A COMPENSACIÓN FAMILIAR;Administradora CF;(.*?);'),
    'f_afiliacion_cf': re.compile(r'AFILIACIÓN A COMPENSACIÓN FAMILIAR;Fecha de Afiliación;(.*?);'),
    'est_afiliacion_cf': re.compile(r'AFILIACIÓN A COMPENSACIÓN FAMILIAR;Estado de Afiliación;(.*?);'),
    'tipo_miembro_pobl_cf': re.compile(r'AFILIACIÓN A COMPENSACIÓN FAMILIAR;Tipo de Miembro de la Población Cubierta;(.*?);'),
    'tipo_afiliado_cf': re.compile(r'AFILIACIÓN A COMPENSACIÓN FAMILIAR;Tipo de Afiliado;(.*?);'),
    'municipio_laboral_cf': re.compile(r'AFILIACIÓN A COMPENSACIÓN FAMILIAR;Municipio Labora;(.*?);'),
    'admon_cesantias': re.compile(r'AFILIACIÓN A CESANTÍAS;Administradora;(.*?);'),
    'f_afiliacion_cesantias': re.compile(r'AFILIACIÓN A CESANTÍAS;Fecha de Afiliación;(.*?);'),
    'est_afiliacion_cesantias': re.compile(r'AFILIACIÓN A CESANTÍAS;Estado de Afiliación;(.*?);'),
    'regimen_cesantias': re.compile(r'AFILIACIÓN A CESANTÍAS;Régimen;(.*?);'),
    'municipio_laboral_cesantias': re.compile(r'AFILIACIÓN A CESANTÍAS;Municipio Labora;(.*?);'),
    'entidad_pension': re.compile(r'PENSIONADOS;Entidad que reconoce la pensión;(.*?);'),
    'f_resolucion_pension': re.compile(r'PENSIONADOS;Fecha Resolución;(.*?);'),
    'est_pension': re.compile(r'PENSIONADOS;Estado;(.*?);'),
    'modalidad_pension': re.compile(r'PENSIONADOS;Modalidad;(.*?);'),
    'num_resolucion_pension': re.compile(r'PENSIONADOS;Número Resolución Pensión PG;(.*?);'),
    'tipo_pension': re.compile(r'PENSIONADOS;Tipo de Pensión;(.*?);'),
    'tipo_pensionado': re.compile(r'PENSIONADOS;Tipo de Pensionado;(.*?);'),
    'admon_vpas': re.compile(r'VINCULACIÓN A PROGRAMAS DE ASISTENCIA SOCIAL;Administradora;(.*?);'),
    'f_vinculacion_vpas': re.compile(r'VINCULACIÓN A PROGRAMAS DE ASISTENCIA SOCIAL;Fecha de Vinculación;(.*?);'),
    'est_vinculacion_vpas': re.compile(r'VINCULACIÓN A PROGRAMAS DE ASISTENCIA SOCIAL;Estado de la Vinculación;(.*?);'),
    'est_beneficio_vpas': re.compile(r'VINCULACIÓN A PROGRAMAS DE ASISTENCIA SOCIAL;Estado del Beneficio;(.*?);'),
    'f_ult_beneficio_vpas': re.compile(r'VINCULACIÓN A PROGRAMAS DE ASISTENCIA SOCIAL;Fecha Ultimo Beneficio;(.*?);'),
    'programa_vpas': re.compile(r'VINCULACIÓN A PROGRAMAS DE ASISTENCIA SOCIAL;Programa;(.*?);'),
    'ubicacion_entrega_vpas': re.compile(r'VINCULACIÓN A PROGRAMAS DE ASISTENCIA SOCIAL;Ubicación de Entrega del Beneficio;(.*?);'),
    'marca_sin_informacion': None  # Este campo es calculado
}

In [125]:
#5. Procesamos los registros para extraer la info y crear el df
datos_estructurados = []
for registro in registros:
    datos = {
        'origen': 'RUAF',  # Asigna 'RUAF' directamente
        'marca_sin_informacion': 1  # Por defecto, marca sin información
    }
    
    for line in registro:
        for campo, patron in patrones.items():
            if patron is not None:  # Asegúrate de que el patrón no sea None
                m = patron.search(line)
                if m:
                    datos[campo] = m.group(1)
                    datos['marca_sin_informacion'] = 0  # Hay información
            else:
                # Aquí puedes manejar campos que no tienen un patrón asociado,
                # como 'origen', que ya está establecido, o cálculos específicos
                pass

    # Asegúrate de agregar valores nulos para las claves que no se llenaron
    for campo in patrones.keys():
        if campo not in datos:
            datos[campo] = None
    
    datos_estructurados.append(datos)

df = pd.DataFrame(datos_estructurados)

In [127]:
df

,origen,marca_sin_informacion,segundo_apellido,primer_apellido,segundo_nombre,primer_nombre,tipo_id_str,dep_municipio,tipo_afiliado_salud,est_afiliacion_salud,...,num_resolucion_pension,tipo_pension,tipo_pensionado,admon_vpas,f_vinculacion_vpas,est_vinculacion_vpas,est_beneficio_vpas,f_ult_beneficio_vpas,programa_vpas,ubicacion_entrega_vpas
0,RUAF,0,M,BEDOYA,MEJIA,MAURICIO,CC,FLORENCIA,COTIZANTE,Activo,...,None,None,None,None,None,None,None,None,None,None
1,RUAF,0,ARIAS,GOMEZ,ANDRES,DAIRO,CC,MEDELLIN,COTIZANTE,Activo,...,None,None,None,None,None,None,None,None,None,None
2,RUAF,0,LEON,OLAYA,DAVID,CRISTIAM,CC,BOGOTA D.C.,COTIZANTE,Activo por emergencia,...,None,None,None,None,None,None,None,None,None,None
3,RUAF,0,GODOY,SOLANO,GERARDO,LUIS,CC,BOGOTA D.C.,BENEFICIARIO,Activo,...,None,None,None,None,None,None,None,None,None,None
4,RUAF,0,URIBE,NARANJO,CRISTINA,ISABEL,CC,BELLO,COTIZANTE,Activo,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
836,RUAF,0,DE LA ASUNCION,TORRES,ALFREDO,BRANDON,CC,BARRANQUILLA,COTIZANTE,Activo,...,None,None,None,None,None,None,None,None,None,None
837,RUAF,0,VARGAS,RUEDA,JOSE,BRAYAN,CC,BARRANQUILLA,CABEZA DE FAMILIA,Activo,...,None,None,None,None,None,None,None,None,None,None
838,RUAF,0,F,ROJAS,BURBANO,DANIELA,CC,SANTANDER DE QUILICHAO,COTIZANTE,Activo,...,None,Sobrevivencia temporal riesgo común,Régimen de ahorro individual. No aplica tope m...,None,None,None,None,None,None,None
839,RUAF,0,M,SEPULVEDA,GALVIS,HILDEBRANDO,CC,MADRID,COTIZANTE,Activo,...,None,None,None,None,None,None,None,None,None,None
